In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../modules/'))
sys.path.insert(1, os.path.join(sys.path[0], '../data_creation/'))
sys.path.insert(1, os.path.join(sys.path[0], '../networks/'))
sys.path.insert(1, os.path.join(sys.path[0], '../network_trainers/'))

from Data_Creator_C import Data_Creator_C as Data_Creator
from CNN_DS_BN_C import CNN_DS_BN_C
from CNN_C_Trainer import CNN_C_Trainer

from data_manipulation import *
import tensorflow as tf

In [2]:
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

In [3]:
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

In [16]:
network = CNN_DS_BN_C('CNN_DS_BN_C_2_161_aug6h_j', 3, 161)
network.print_params()

{'adam_initial_learning_rate': 0.0001,
 'log_dir': '../logs/',
 'name': 'CNN_DS_BN_C_2_161_aug6h_j',
 'num_classes': 161,
 'num_downsamples': 3}


In [17]:
trainer = CNN_C_Trainer(network,
                        Data_Creator,
                        161,
                        num_epochs = 500,
                        batch_size = 30,
                        model_save_interval = 10,
                        conv_keep_prob = 0.90,
                        sample_keep_prob = 1,
                        single_dataset = False)
trainer.print_params()

{'batch_size': 30,
 'conv_keep_prob': 0.9,
 'log_dir': '../logs/',
 'metric_names': ['costs', 'accuracies'],
 'model_save_interval': 10,
 'name': 'CNN_DS_BN_C_2_161_aug6h_j',
 'num_classes': 161,
 'num_epochs': 500,
 'pretrained_model_path': None,
 'sample_keep_prob': 1,
 'single_dataset': False}


In [18]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains,
                 num_flatnesses = 50,
                 precision = 0.00025,
                 blur = 0.10)

In [19]:
trainer.train()

saving metrics - saved

In [24]:
network = CNN_DS_BN_C('CNN_DS_BN_C_2_161_aug6h_k', 3, 161)
network.print_params()

{'adam_initial_learning_rate': 0.0001,
 'log_dir': '../logs/',
 'name': 'CNN_DS_BN_C_2_161_aug6h_k',
 'num_classes': 161,
 'num_downsamples': 3}


In [25]:
trainer = CNN_C_Trainer(network,
                        Data_Creator,
                        161,
                        num_epochs = 10,
                        batch_size = 30,
                        model_save_interval = 10,
                        conv_keep_prob = 0.90,
                        sample_keep_prob = 1,
                        single_dataset = False,
                        pretrained_model_path = '../logs/CNN_DS_BN_C_2_161_aug6h_j/trained_model.ckpt-500')
trainer.print_params()

{'batch_size': 30,
 'conv_keep_prob': 0.9,
 'log_dir': '../logs/',
 'metric_names': ['costs', 'accuracies'],
 'model_save_interval': 10,
 'name': 'CNN_DS_BN_C_2_161_aug6h_k',
 'num_classes': 161,
 'num_epochs': 10,
 'pretrained_model_path': '../logs/CNN_DS_BN_C_2_161_aug6h_j/trained_model.ckpt-500',
 'sample_keep_prob': 1,
 'single_dataset': False}


In [26]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains,
                 num_flatnesses = 50,
                 precision = 0.00025,
                 blur = 0.20)

In [27]:
trainer.train()

network Ready - 1415085 trainable parametersINFO:tensorflow:Restoring parameters from ../logs/CNN_DS_BN_C_2_161_aug6h_j/trained_model.ckpt-500
saving metrics - saved